# E2


In [237]:
import pandas as pd
import os
os.chdir('/Users/hanwang/Desktop/DataScienceCaseStudy') 
os.getcwd()

'/Users/hanwang/Desktop/DataScienceCaseStudy'

In [238]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import math
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [239]:
train = pd.read_csv("train.csv").drop(['Id'],axis=1)
test = pd.read_csv("test.csv").drop(['Id'],axis=1) 

In [240]:
y=list(train['SalePrice'])
y = np.log1p(y)
all_data = pd.concat((train,
                      test)).drop(['SalePrice'],axis = 1)


In [241]:
all_data['MSSubClass'] = all_data['MSSubClass'].astype(str).astype(object)
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])


In [242]:
Dict = {"No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5,
             "Mn": 2, "Av": 3, 
             "Unf": 1, "LwQ": 2, "Rec": 3, "BLQ": 4, "ALQ": 5, "GLQ": 6,
             "Sal": 1, "Sev": 2, "Maj2": 3, "Maj1": 4, "Mod": 5, "Min2": 6, "Min1": 7, "Typ": 8,
             "RFn": 2, "Fin": 3,
             "MnWw": 1, "GdWo": 2, "MnPrv": 3, "GdPrv": 4,
             "N": 0, "Y": 1, np.nan:0
            }

for col in ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
          'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond',
          'BsmtFinType1', 'BsmtFinType2', 'Functional', 'GarageFinish', 'Fence', 
          'CentralAir']:

        all_data[col] = all_data[col].map(Dict).astype(int)

In [243]:
all_data = all_data.fillna({
    'Alley' : 'NoAlley',
    'MasVnrType': 'None',
    'FireplaceQu': 'NoFireplace',
    'GarageType': 'NoGarage',
    'GarageFinish': 'NoGarage',
    'GarageQual': 'NoGarage',
    'GarageCond': 'NoGarage',
    'BsmtFullBath': 0,
    'BsmtHalfBath': 0,
    'BsmtQual' : 'NoBsmt',
    'BsmtCond' : 'NoBsmt',
    'BsmtExposure' : 'NoBsmt',
    'BsmtFinType1' : 'NoBsmt',
    'BsmtFinType2' : 'NoBsmt',
    'KitchenQual' : 'TA',
    'MSZoning' : 'RL',
    'Utilities' : 'AllPub',
    'Exterior1st' : 'VinylSd',
    'Exterior2nd'   : 'VinylSd',
    'Functional' : 'Typ',
    'PoolQC' : 'NoPool',
    'Fence' : 'NoFence',
    'MiscFeature' : 'None',
    'Electrical' : 'SBrkr'
     
})


all_data.loc[all_data.SaleCondition.isnull(), 'SaleCondition'] = 'Normal'
all_data.loc[all_data.SaleCondition.isnull(), 'SaleType'] = 'WD'
all_data.loc[all_data.MasVnrType == 'None', 'MasVnrArea'] = 0

all_data.loc[all_data.BsmtFinSF1.isnull(), 'BsmtFinSF1'] = all_data.BsmtFinSF1.median()
all_data.loc[all_data.BsmtUnfSF.isnull(), 'BsmtUnfSF'] = all_data.BsmtUnfSF.median()






# only one is null and it has type Detchd
all_data.loc[all_data['GarageArea'].isnull(), 'GarageArea'] = all_data.loc[all_data['GarageType']=='Detchd', 'GarageArea'].mean()
all_data.loc[all_data['GarageCars'].isnull(), 'GarageCars'] = all_data.loc[all_data['GarageType']=='Detchd', 'GarageCars'].median()

In [244]:
all_data = all_data.drop(['Street','Utilities','Condition2','RoofMatl',
                          'LowQualFinSF','3SsnPorch','PoolArea','PoolQC','MiscVal','GarageYrBlt'],axis = 1)

In [245]:
#warnings.simplefilter('ignore', np.RankWarning)
# I have no idea how to do it better. Probably, it is better to do nothing
a1 = all_data.loc[np.logical_not(all_data["LotFrontage"].isnull()), "LotArea"]
a2 = all_data.loc[np.logical_not(all_data["LotFrontage"].isnull()), "LotFrontage"]
# plt.scatter(x, y)
t = (a1 <= 25000) & (a2 <= 150)
p = np.polyfit(a1[t], a2[t], 1)
all_data.loc[all_data['LotFrontage'].isnull(), 'LotFrontage'] = np.polyval(p, all_data.loc[all_data['LotFrontage'].isnull(), 'LotArea'])

In [246]:
#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
prices = pd.DataFrame({"price":train["SalePrice"], "log(price + 1)":np.log1p(train["SalePrice"])})
all_data = all_data.replace(0,all_data.mean())
from numpy import inf
all_data = all_data.replace(-inf , 0)
all_data = all_data.replace(inf , 0)

In [247]:
all_data = pd.get_dummies(all_data)

In [248]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = np.log1p(train["SalePrice"])

In [249]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
import sklearn
assert(sklearn.__version__ > '0.18' and sklearn.__version__ < '0.20')
import warnings
import sklearn.model_selection as ms

warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
%matplotlib inline
import sklearn.model_selection as ms
from sklearn import linear_model

x=X_train 



## Elastic Net


In [255]:
from sklearn.linear_model import ElasticNet
en = ElasticNet(random_state=123456)

In [303]:
import numpy as np
from sklearn.grid_search import GridSearchCV

param_grid_1 = {'alpha': np.arange(0.01,1,0.01),'l1_ratio':[0.02]}
grid_en= GridSearchCV(en, param_grid_1,cv =10, scoring = 'neg_mean_squared_error',verbose = 2)


In [304]:
grid_en.fit(x,y)

Fitting 10 folds for each of 99 candidates, totalling 990 fits
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.01, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.01, l1_ratio=0.02 -   0.0s
[CV] alpha=0.02, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.02, l1_ratio=0.02 -   0.0s
[CV] alpha=0.02, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.02, l1_ratio=0.02 -   0.0s
[CV] alpha=0.02, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.02, l1_ratio=0.02 -   0.0s
[CV] a

[CV] .............................. alpha=0.07, l1_ratio=0.02 -   0.0s
[CV] alpha=0.07, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.07, l1_ratio=0.02 -   0.0s
[CV] alpha=0.07, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.07, l1_ratio=0.02 -   0.0s
[CV] alpha=0.07, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.07, l1_ratio=0.02 -   0.0s
[CV] alpha=0.07, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.07, l1_ratio=0.02 -   0.0s
[CV] alpha=0.08, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.08, l1_ratio=0.02 -   0.0s
[CV] alpha=0.08, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.08, l1_ratio=0.02 -   0.0s
[CV] alpha=0.08, l1_ratio=0.02 .......................................
[CV] .

[CV] .............................. alpha=0.14, l1_ratio=0.02 -   0.0s
[CV] alpha=0.14, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.14, l1_ratio=0.02 -   0.0s
[CV] alpha=0.14, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.14, l1_ratio=0.02 -   0.0s
[CV] alpha=0.14, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.14, l1_ratio=0.02 -   0.0s
[CV] alpha=0.14, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.14, l1_ratio=0.02 -   0.0s
[CV] alpha=0.14, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.14, l1_ratio=0.02 -   0.0s
[CV] alpha=0.14, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.14, l1_ratio=0.02 -   0.0s
[CV] alpha=0.14, l1_ratio=0.02 .......................................
[CV] .

...

[CV] .............................. alpha=0.84, l1_ratio=0.02 -   0.0s
[CV] alpha=0.84, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.84, l1_ratio=0.02 -   0.0s
[CV] alpha=0.84, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.84, l1_ratio=0.02 -   0.0s
[CV] alpha=0.84, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.84, l1_ratio=0.02 -   0.0s
[CV] alpha=0.84, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.84, l1_ratio=0.02 -   0.0s
[CV] alpha=0.84, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.84, l1_ratio=0.02 -   0.0s
[CV] alpha=0.84, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.84, l1_ratio=0.02 -   0.0s
[CV] alpha=0.85, l1_ratio=0.02 .......................................
[CV] .

[CV] ............................... alpha=0.9, l1_ratio=0.02 -   0.0s
[CV] alpha=0.9, l1_ratio=0.02 ........................................
[CV] ............................... alpha=0.9, l1_ratio=0.02 -   0.0s
[CV] alpha=0.9, l1_ratio=0.02 ........................................
[CV] ............................... alpha=0.9, l1_ratio=0.02 -   0.0s
[CV] alpha=0.9, l1_ratio=0.02 ........................................
[CV] ............................... alpha=0.9, l1_ratio=0.02 -   0.0s
[CV] alpha=0.9, l1_ratio=0.02 ........................................
[CV] ............................... alpha=0.9, l1_ratio=0.02 -   0.0s
[CV] alpha=0.9, l1_ratio=0.02 ........................................
[CV] ............................... alpha=0.9, l1_ratio=0.02 -   0.0s
[CV] alpha=0.91, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.91, l1_ratio=0.02 -   0.0s
[CV] alpha=0.91, l1_ratio=0.02 .......................................
[CV] .

[CV] .............................. alpha=0.97, l1_ratio=0.02 -   0.0s
[CV] alpha=0.97, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.97, l1_ratio=0.02 -   0.0s
[CV] alpha=0.97, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.97, l1_ratio=0.02 -   0.0s
[CV] alpha=0.97, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.97, l1_ratio=0.02 -   0.0s
[CV] alpha=0.97, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.97, l1_ratio=0.02 -   0.0s
[CV] alpha=0.97, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.97, l1_ratio=0.02 -   0.0s
[CV] alpha=0.98, l1_ratio=0.02 .......................................
[CV] .............................. alpha=0.98, l1_ratio=0.02 -   0.0s
[CV] alpha=0.98, l1_ratio=0.02 .......................................
[CV] .

[Parallel(n_jobs=1)]: Done 990 out of 990 | elapsed:   18.4s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=123456, selection='cyclic', tol=0.0001,
      warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.01,  0.02, ...,  0.98,  0.99]), 'l1_ratio': [0.02]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=2)

In [305]:
from math import sqrt
sqrt(-grid_en.best_score_)

0.1255488863184298

In [306]:
grid_en.best_estimator_

ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True, l1_ratio=0.02,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=123456, selection='cyclic', tol=0.0001,
      warm_start=False)

# Lasso

In [262]:
import numpy as np
param_grid = {'alpha': np.arange(0.0001,0.01,0.0001)}
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model
lasso = linear_model.Lasso()
grid_clf = GridSearchCV(lasso, param_grid, cv =10, scoring = 'neg_mean_squared_error',verbose = 2)

In [263]:
grid_clf.fit(x,y)


GridSearchCV(cv=10, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001,  0.0002, ...,  0.0098,  0.0099])},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

In [264]:
grid_clf.best_estimator_

Lasso(alpha=0.00050000000000000001, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [265]:
from math import sqrt
sqrt(-grid_clf.best_score_)

0.12332225073032545

## Ridge


In [271]:
from sklearn import linear_model
ridge = linear_model.Ridge()
param_grid_2 = {'alpha': np.arange(0.1,1,0.01)}
grid_rd = GridSearchCV(ridge, param_grid_2, cv =10, scoring = 'neg_mean_squared_error',verbose = 2)
grid_rd.fit(x, y)

Fitting 10 folds for each of 90 candidates, totalling 900 fits
[CV] alpha=0.1 .......................................................
[CV] .............................................. alpha=0.1 -   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............................................. alpha=0.1 -   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............................................. alpha=0.1 -   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............................................. alpha=0.1 -   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............................................. alpha=0.1 -   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............................................. alpha=0.1 -   0.0s
[CV] alpha=0.1 .......................................................
[CV] .........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] .............................................. alpha=0.1 -   0.0s
[CV] alpha=0.11 ......................................................
[CV] ............................................. alpha=0.11 -   0.0s
[CV] alpha=0.11 ......................................................
[CV] ............................................. alpha=0.11 -   0.0s
[CV] alpha=0.11 ......................................................
[CV] ............................................. alpha=0.11 -   0.0s
[CV] alpha=0.11 ......................................................
[CV] ............................................. alpha=0.11 -   0.0s
[CV] alpha=0.11 ......................................................
[CV] ............................................. alpha=0.11 -   0.0s
[CV] alpha=0.11 ......................................................
[CV] ............................................. alpha=0.11 -   0.0s
[CV] alpha=0.11 ......................................................
[CV] .

[CV] ............................................. alpha=0.17 -   0.0s
[CV] alpha=0.17 ......................................................
[CV] ............................................. alpha=0.17 -   0.0s
[CV] alpha=0.17 ......................................................
[CV] ............................................. alpha=0.17 -   0.0s
[CV] alpha=0.17 ......................................................
[CV] ............................................. alpha=0.17 -   0.0s
[CV] alpha=0.17 ......................................................
[CV] ............................................. alpha=0.17 -   0.0s
[CV] alpha=0.17 ......................................................
[CV] ............................................. alpha=0.17 -   0.0s
[CV] alpha=0.17 ......................................................
[CV] ............................................. alpha=0.17 -   0.0s
[CV] alpha=0.17 ......................................................
[CV] .

[CV] ............................................. alpha=0.23 -   0.0s
[CV] alpha=0.23 ......................................................
[CV] ............................................. alpha=0.23 -   0.0s
[CV] alpha=0.23 ......................................................
[CV] ............................................. alpha=0.23 -   0.0s
[CV] alpha=0.24 ......................................................
[CV] ............................................. alpha=0.24 -   0.0s
[CV] alpha=0.24 ......................................................
[CV] ............................................. alpha=0.24 -   0.0s
[CV] alpha=0.24 ......................................................
[CV] ............................................. alpha=0.24 -   0.0s
[CV] alpha=0.24 ......................................................
[CV] ............................................. alpha=0.24 -   0.0s
[CV] alpha=0.24 ......................................................
[CV] .

...

[CV] ............................................. alpha=0.81 -   0.0s
[CV] alpha=0.82 ......................................................
[CV] ............................................. alpha=0.82 -   0.0s
[CV] alpha=0.82 ......................................................
[CV] ............................................. alpha=0.82 -   0.0s
[CV] alpha=0.82 ......................................................
[CV] ............................................. alpha=0.82 -   0.0s
[CV] alpha=0.82 ......................................................
[CV] ............................................. alpha=0.82 -   0.0s
[CV] alpha=0.82 ......................................................
[CV] ............................................. alpha=0.82 -   0.0s
[CV] alpha=0.82 ......................................................
[CV] ............................................. alpha=0.82 -   0.0s
[CV] alpha=0.82 ......................................................
[CV] .

[CV] ............................................. alpha=0.87 -   0.0s
[CV] alpha=0.88 ......................................................
[CV] ............................................. alpha=0.88 -   0.0s
[CV] alpha=0.88 ......................................................
[CV] ............................................. alpha=0.88 -   0.0s
[CV] alpha=0.88 ......................................................
[CV] ............................................. alpha=0.88 -   0.0s
[CV] alpha=0.88 ......................................................
[CV] ............................................. alpha=0.88 -   0.0s
[CV] alpha=0.88 ......................................................
[CV] ............................................. alpha=0.88 -   0.0s
[CV] alpha=0.88 ......................................................
[CV] ............................................. alpha=0.88 -   0.0s
[CV] alpha=0.88 ......................................................
[CV] .

[CV] ............................................. alpha=0.94 -   0.0s
[CV] alpha=0.94 ......................................................
[CV] ............................................. alpha=0.94 -   0.0s
[CV] alpha=0.94 ......................................................
[CV] ............................................. alpha=0.94 -   0.0s
[CV] alpha=0.94 ......................................................
[CV] ............................................. alpha=0.94 -   0.0s
[CV] alpha=0.94 ......................................................
[CV] ............................................. alpha=0.94 -   0.0s
[CV] alpha=0.94 ......................................................
[CV] ............................................. alpha=0.94 -   0.0s
[CV] alpha=0.94 ......................................................
[CV] ............................................. alpha=0.94 -   0.0s
[CV] alpha=0.95 ......................................................
[CV] .

[Parallel(n_jobs=1)]: Done 900 out of 900 | elapsed:   19.0s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.1 ,  0.11, ...,  0.98,  0.99])},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=2)

Ridge(alpha=0.98999999999999955, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)

In [273]:
from math import sqrt
sqrt(-grid_rd.best_score_)

0.12834505404934946

##  Gradient Boosting


In [296]:
from sklearn import ensemble
gbm= ensemble.GradientBoostingRegressor(random_state = 1)



In [297]:
params = {'max_features': ['sqrt'],
          'min_samples_split': [10],
          'min_samples_leaf': [2],
          'max_depth': [4],
          'n_estimators':[3000],
          'learning_rate':[0.01],
          'subsample':[0.7],
          'loss':['ls'],
          }
grid_gbm = GridSearchCV(gbm, params, cv =10, scoring = 'neg_mean_squared_error',verbose = 2)

grid_gbm.fit(x,y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 
[CV]  learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 -   5.0s
[CV] learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s


[CV]  learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 -   4.5s
[CV] learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 
[CV]  learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 -   4.6s
[CV] learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 
[CV]  learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 -   4.4s
[CV] learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=3000, subsample=0.7 
[CV]  learning_rate=0.01, loss=ls, max_depth=4, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   43.7s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=1,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': ['sqrt'], 'min_samples_split': [10], 'min_samples_leaf': [2], 'max_depth': [4], 'n_estimators': [3000], 'learning_rate': [0.01], 'subsample': [0.7], 'loss': ['ls']},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=2)

In [298]:
from math import sqrt
sqrt(-grid_rd.best_score_)

0.12834505404934946

##  Random Forest


In [307]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 0)
param_grid = {'n_estimators': [3000],
               'max_features': ['sqrt'],
               'max_depth': [21],
               'min_samples_split': [2],
               'min_samples_leaf': [1],
               'bootstrap' : [False]
               }

grid_rf = GridSearchCV(estimator = rf, param_grid = param_grid,
                           cv = 10, n_jobs = -1, verbose = 2, scoring = 'neg_mean_squared_error')

grid_rf.fit(x,y)


Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=3000 
[CV] bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=3000 
[CV] bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=3000 
[CV] bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=3000 
[CV]  bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=3000 -  40.2s
[CV]  bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=3000 -  40.2s
[CV]  bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=3000 -  40.2s
[CV] bootstrap=False, max_depth=21, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimator

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [3000], 'max_features': ['sqrt'], 'max_depth': [21], 'min_samples_split': [2], 'min_samples_leaf': [1], 'bootstrap': [False]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=2)

In [308]:
from math import sqrt
sqrt(-grid_rf.best_score_)

0.1362341635356765